In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
from datetime import datetime
# from sklearn.cluster import KMeans
from scipy.stats import norm
import time
import os
from pylab import rcParams

from module_analysis import *

rcParams['figure.figsize'] = 10, 7.5
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 30}
plt.rc('font', **font)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#Define the type of analysis and input files

In [9]:
analysis_type='sick' # 'sick' or 'vaccinated'(Define the type of stress event to analyse)
subgroup_type='full' ## 'full'/'IRIS' (Define IRIS class full: all RVI subjects or IR versus IS comparision)
# fil_data="/Users/Files/InputFile.txt"
# fil_annotation="/Users/Files/Annotation.txt"
if_log_transformed=False # specify if the data has been log-transformed. if so, transform it back in the bottom

# Do not edit the things below

In [10]:
################ put all the configuration information here ################
## Parameter setting 
FDR = 0.1
daytol = 100 # determine the pre- and post- infection
debug_mode = False ## visualization
if_normalize=True # True/False. If True, normalize the data.
sample_balance_percentage=0.3 ## one patient can only have less than sample_balance_percentage samples in the group
win_size=186 ## use a two-sided window with one side sidth 183 (all together 366) 

In [11]:
################ the parameters in this blockare automatically generated ################
## analysis type
time_stamp=time.strftime("%m_%d_%Y_%H_%M")
output_folder="./analysis_"+analysis_type+'_'+subgroup_type+'_'+time_stamp
# output_folder="./debug"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if analysis_type=='sick':
    TT_list=['Pre1','Infection_Early','Infection_Middle','Infection_Late',\
             'Infection_Recovery_Early','Infection_Recovery_Late','Post1','Else']
elif analysis_type=='vaccinated':
    TT_list=['Pre1','Imz_Early','Imz_Middle','Imz_Late',\
             'Imz_Recovery_Early','Imz_Recovery_Late','Post1','Else']
elif analysis_type=='antibio':
    TT_list=['Pre1','Ant_Early','Ant_Middle','Ant_Late',\
             'Ant_Recovery_Early','Ant_Recovery_Late','Post1','Else']

## group different patients into different groups
group_config=[[0],[1,2],[3],[4,5],[6],[7]]
group_list=['Pre','EARLY','LATE','RECOVERY','Post','Else']

# Import the data

In [12]:
################ data loading ################
print('output_folder: \n',output_folder)

## Data loading and normalization
CountData,MetaData,GeneNames,Header = get_data(fil_data,fil_annotation,\
                                               if_normalize=if_normalize,output_folder=output_folder)
n_sample,n_gene = CountData.shape

## Extract the data label
name_list,labels = get_label(MetaData,TT_list,group_config,daytol=daytol)
label_ID,ID_list = labels['label_ID'],name_list['ID_list']
label_HS,HS_list = labels['label_HS'],name_list['HS_list']
label_IRIS,IRIS_list = labels['label_IRIS'],name_list['IRIS_list']
label_DBC,DBC_list = labels['label_DBC'],name_list['DBC_list']
label_batch,batch_list = labels['label_batch'],name_list['batch_list']
label_date = labels['label_date']
label_TT = labels['label_TT']
label_group = labels['label_group']

## transform back the log transformed data
if if_log_transformed:
    CountData = 10**(CountData)-1

## mean imputation
CountData = impute(CountData,label_ID,label_HS)

## write the metadata
print('### write the metadata\n')
f=open(output_folder+"/metadata.txt",'w')
f.write('SampleID\tSubjectID\tBatch\tDate\tIRIS\tClass\tstate1\ttime-series\n')
for i in range(n_sample):
    for j in range(7):
        f.write(MetaData[i][j])
        f.write('\t')
    f.write(TT_list[label_TT[i]])    
    if i <n_sample-1:
        f.write('\n')
f.close()

## write the normalized count data
print('### write the normalized count data\n')
output_file=output_folder+'/NormalizedCount.txt'
data_write(CountData,MetaData,GeneNames,Header,output_file)

## Sanity check of the labels
print('### sanity check')
for i in range(10):
    print(MetaData[i])
    print(ID_list[label_ID[i]],HS_list[label_HS[i]],IRIS_list[label_IRIS[i]],DBC_list[label_DBC[i]],\
         batch_list[label_batch[i]],label_date[i],TT_list[label_TT[i]],group_list[label_group[i]])
    print('')

output_folder: 
 ./analysis_sick_full_06_24_2018_10_21
### read annotation file: 
header_annotation
['SampleID', 'SubjectID', 'Batch', 'Date', 'IRIS', 'Class', 'state1']


### read data file: 
n_sample=981, n_gene=724

### cross reference the annotation and the data: 


### filtering and normalization
if_normalize: True
Gene filter threshold = 1.00
Sample filter threshold = 1303726.43
### filtering and normalization completed
n_sample=974, n_gene=724

### extract the data label
#  Pre1 :  50
#  Infection_Early :  31
#  Infection_Middle :  49
#  Infection_Late :  31
#  Infection_Recovery_Early :  37
#  Infection_Recovery_Late :  28
#  Post1 :  43
#  Else :  705
### write the metadata

### write the normalized count data

### sanity check
['69-001-02', '69-001', '1', '2-Apr-10', 'IS', 'Diabetic', 'Infection_Middle']
69-001 Infection_Middle IS Diabetic 1 2010-4-2 Infection_Middle EARLY

['69-001-07', '69-001', '1', '1-Oct-10', 'IS', 'Diabetic', 'Healthy']
69-001 Healthy IS Diabetic 1 2010

# Analysis

In [16]:
np.random.seed(35)
config={}
config['FDR']=FDR
config['TT_list']=TT_list
config['Title']='************ analysis: '+analysis_type+'_'+subgroup_type+'************'
config['output_folder']=output_folder
config['group_config']=group_config
config['group_list']=group_list
config['win_size']=win_size
config['sample_balance_percentage']=sample_balance_percentage
config['debug_mode']=debug_mode
analysis_label=[]

if subgroup_type=='full':
    config['label_TT']=label_TT
    config['label_group']=label_group
    config['label_date']=label_date
    config['label_ID']=label_ID
    config['label_HS']=label_HS
    config['suffix']='_full'
    sig_idx,cluster_label,temp=analysis(CountData,GeneNames,config,vis=1)
    analysis_label.append(temp)
    
elif subgroup_type=='IRIS':
    config['suffix']='_IR'
    config['label_TT']=label_TT[label_IRIS==0]
    config['label_group']=label_group[label_IRIS==0]
    config['label_date']=label_date[label_IRIS==0]
    config['label_ID']=label_ID[label_IRIS==0]
    config['label_HS']=label_HS[label_IRIS==0]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_IRIS==0]=analysis(CountData[label_IRIS==0,:],GeneNames,config,vis=1)
    analysis_label.append(temp)
    config['suffix']='_IS'
    config['label_TT']=label_TT[label_IRIS==1]
    config['label_group']=label_group[label_IRIS==1]
    config['label_date']=label_date[label_IRIS==1]
    config['label_ID']=label_ID[label_IRIS==1]
    config['label_HS']=label_HS[label_IRIS==1]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_IRIS==1]=analysis(CountData[label_IRIS==1,:],GeneNames,config,vis=1)    
    analysis_label.append(temp)
     
elif subgroup_type=='DBC':
    config['suffix']='_Diabetic'
    config['label_TT']=label_TT[label_DBC==0]
    config['label_group']=label_group[label_DBC==0]
    config['label_date']=label_date[label_DBC==0]
    config['label_ID']=label_ID[label_DBC==0]
    config['label_HS']=label_HS[label_DBC==0]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_DBC==0]=analysis(CountData[label_DBC==0,:],GeneNames,config,vis=1)
    analysis_label.append(temp)
    config['suffix']='_Violator'
    config['label_TT']=label_TT[label_DBC==2]
    config['label_group']=label_group[label_DBC==2]
    config['label_date']=label_date[label_DBC==2]
    config['label_ID']=label_ID[label_DBC==2]
    config['label_HS']=label_HS[label_DBC==2]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_DBC==2]=analysis(CountData[label_DBC==2,:],GeneNames,config,vis=1)
    analysis_label.append(temp)
    config['suffix']='_Prediabetic'
    config['label_TT']=label_TT[label_DBC==3]
    config['label_group']=label_group[label_DBC==3]
    config['label_date']=label_date[label_DBC==3]
    config['label_ID']=label_ID[label_DBC==3]
    config['label_HS']=label_HS[label_DBC==3]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_DBC==3]=analysis(CountData[label_DBC==3,:],GeneNames,config,vis=1)
    analysis_label.append(temp)
    config['suffix']='_Control'
    config['label_TT']=label_TT[label_DBC==4]
    config['label_group']=label_group[label_DBC==4]
    config['label_date']=label_date[label_DBC==4]
    config['label_ID']=label_ID[label_DBC==4]
    config['label_HS']=label_HS[label_DBC==4]
    temp=np.zeros([n_sample],dtype=int)+len(group_list)-1
    sig_idx,cluster_label,temp[label_DBC==4]=analysis(CountData[label_DBC==4,:],GeneNames,config,vis=1)
    analysis_label.append(temp)
    
## write the metadata with paired_AUC analysis information
if sig_idx is not None:
    if subgroup_type=='full':
        analysis_list=['full']
    elif subgroup_type=='IRIS':
        analysis_list=['IR','IS']
    elif subgroup_type=='DBC':
        analysis_list=['Diabetic', 'Violator', 'Prediabetic', 'Control']
    f=open(output_folder+"/metadata_analysis.txt",'w')
    f.write('SampleID\tSubjectID\tBatch\tDate\tIRIS\tClass\tstate1\ttime-series\t')
    for j in range(len(analysis_list)):
        f.write('TT_'+analysis_list[j]+'\t')
    f.write('\n')
    for i in range(n_sample):
        for j in range(7):
            f.write(MetaData[i][j])
            f.write('\t')
        f.write(TT_list[label_TT[i]]+'\t')  
        for j in range(len(analysis_list)):
            f.write(group_list[analysis_label[j][i]])
            f.write('\t')
        if i <n_sample-1:
            f.write('\n')
    f.close()